## Integração de dados

#### Masterlens, CASTLES

In [1]:
import pandas as pd
from astropy.coordinates import Angle

In [2]:
# dec: -27:52:25.7 ---> -27.873806 deg
def sexdeg_to_deg(coord):
    result = Angle(coord + " degrees").degree
    return result #*3600


In [3]:
# ra: 00h49m41.89s ---> 12.424542 deg
def hour_to_deg(coord):  # string coord
    result = Angle(coord).degree
    return result #*3600

In [4]:
# ra: 00:49:41.89 ---> 00h49m41.89s
def hour(coord):
    result = coord.replace(":", 'h', 1)
    result = result.replace(":", 'm', 1)
    result += 's'
    return result

In [5]:
# system table ---------->  143 lenses
def system():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['Name', 'RA(deg)_c', 'Dec(deg)_c', 'E(B-V)_c', 'FGHz (mJy)_c', 'Nimages_c',
                     'size(arccsec)_c', 'sigma (km/s)_c']
    castles = castles.set_index('Name')
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):
        
        # getting data from system file
        ml = pd.read_csv('Scraping/results/system_{}.csv'.format(i))

        # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                       'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                       'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]

        # getting coords from coords file
        coords = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    all_ml = all_ml.set_index('Name')

    # joining castles + ml data
    result = pd.merge(castles, all_ml, how ='outer', on='Name')  


    return result
        


In [37]:
cats = system()
cats.reset_index(inplace=True)
#cats.loc[[130,42], : ]

In [41]:
castles = pd.read_csv('Scraping/castles.csv')
castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
castles.columns=['Name', 'RA(deg)_c', 'Dec(deg)_c', 'E(B-V)_c', 'FGHz (mJy)_c', 'Nimages_c',
                     'size(arccsec)_c', 'sigma (km/s)_c']
castles.loc[42, :]

Name               FSC10214+4724
RA(deg)_c                156.144
Dec(deg)_c               47.1531
E(B-V)_c                   0.012
FGHz (mJy)_c                 NaN
Nimages_c                     2E
size(arccsec)_c             1.59
sigma (km/s)_c               NaN
Name: 42, dtype: object

In [ ]:
# comparing 'alternate names' with 'name'      partial merge?

for index, row in cats.iterrows():
    if type(row['Alternate Names']) == str:
        row['Alternate Names'] = row['Alternate Names'].split(', ')  # list with alternate names
        for i in row['Alternate Names']:
            if index == i:
                print('problem')


In [44]:
# system table  ------------> 138 clustres
def system_cluster():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles = castles[['RA (J2000)', 'Dec (J2000)' ]]
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['ra', 'dec']  # degrees
    
    castles['idCatalog'] = 1
    #castles = castles.set_index('Name')
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):

        # getting coords from coords file
        ml = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i))
        ml = ml[['RA [°]', 'Dec [°]']]
        ml.columns=['ra', 'dec']  # degrees
        ml['idCatalog'] = 2

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    
    # concatenating both catalogs to use Nacluster
    final = pd.concat([castles,all_ml])
    final.reset_index(drop = True, inplace=True)
    final = final.sample(frac=1)  # shuffling dataframe

    return final
        


In [45]:
system_na = system_cluster()
system_na.to_csv('catalogs.csv', header=False)

In [46]:
system_na

,ra,dec,idCatalog
93,319.211458,2.429694,1
120,36.296005,-4.909300,2
25,127.923292,52.754722,1
4,22.805854,43.970317,1
89,306.543458,-45.607528,1
92,317.725083,21.516861,1
18,109.014917,47.147222,1
107,12.424747,-27.874020,2
136,136.033203,-0.998014,2
32,147.844042,26.587250,1


In [47]:
system_na.loc[[130,42], : ]

,ra,dec,idCatalog
130,114.401667,32.261194,2
42,156.144167,47.153056,1


In [48]:
pd.read_csv("clusters_catalogs.csv")

ParserError: Error tokenizing data. C error: Expected 3 fields in line 3, saw 4


In [10]:
# Scraping Joao
# lens objects  -  227
castles2 = pd.read_csv('CastelLensData.csv')
castles2 = castles2.set_index('lens_names')

HE_c = castles2.loc[ 'HE0435-1223' , : ]
HE_c

,ra,ra_err,dec,dec_err,description
lens_names,,,,,
HE0435-1223,4.63747,0.000,-12.2873,0.000,deflected
HE0435-1223,3.16147,0.003,-11.7343,0.003,deflected
HE0435-1223,2.17047,0.003,-12.8903,0.005,deflected
HE0435-1223,3.69847,0.003,-13.9013,0.003,deflected
HE0435-1223,3.47247,0.003,-12.8603,0.003,galaxy


In [11]:
# Object table
HE_ml = pd.read_csv('Scraping/results/redshift_28.csv')
HE_ml.drop('Unnamed: 1', axis=1, inplace=True)
HE_ml.rename(columns={"Unnamed: 0": "Description"})

,Description,Mag,No.,z
0,Lens,NaN,1,0.460
1,Source,18.3,1,1.689
2,Source,18.2,2,1.689
3,Source,18.2,3,1.689
4,Source,18.8,4,1.689
